In [ ]:
import pandas as pd
import json
from bs4 import BeautifulSoup, SoupStrainer
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import requests
import pandas as pd
import requests
import json
import time


Scrapping Car listing URls

In [ ]:
"""
Url Details
rcp = no of listing on page
rcs = no. of pages
    if rcs 100 and rcp = 100 it will start from page 2 i.e. it will start from result 201
    if rcs 200 and rcp = 100 it will start from page 3 i.e. it will start from result 301
sts=New-Used
    if New-Used searches for new, used and certified pre owned
    if Used only used and certified pre-owned

prv = state i.e. Ontario capital case

prx = 100 100 km
        -2 provincial
        prx -1 national
        
srt = 3 high to low
      4 low to high

loc = city case do no matter

adtype = Private only private seller
         Dealer only dealer
         none both
    
basr_url = https://www.autotrader.ca/cars/on/toronto/?rcp=15&rcs=0&srt=3&prx=100&prv=Ontario&loc=toronto&hprc=True&wcp=False&sts=Used&adtype=Private&inMarket=advancedSearch

location = toronto
distancs = +100
condition = used and certified pre owned
seller type = private
other option = with price
"""

In [ ]:
# location= toronto dealer = private
rcp = 100
rcs =0 # add 100 after every iteration
carTitle = []
linkUrl = []

base_url = "https://www.autotrader.ca/cars/?rcp=100&rcs={}&srt=3&prx=-1&loc=toronto&hprc=True&wcp=False&sts=Used&adtype=Private&inMarket=advancedSearch".format(rcs)
headers={'User-Agent': 'Mozilla/5.0'}
response = requests.get(base_url, headers = headers)
soup = BeautifulSoup(response.text, "html.parser")
noOfResult = soup.find_all("h1", class_="rfTitle-wrapper")

for inr in noOfResult:
    countResultstr = inr.find("span")
print("Search Results: {}\n"+format(countResultstr.text))

countResults = int(countResultstr.text.replace(',', ''))/100

for i in range(int(countResults)):
    base_url =   "https://www.autotrader.ca/cars/?rcp=100&rcs={}&srt=3&prx=-1&loc=toronto&hprc=True&wcp=False&sts=Used&adtype=Private&inMarket=advancedSearch".format(rcs)
    headers={'User-Agent': 'Mozilla/5.0'}
    response = requests.get(base_url, headers = headers)
    soup = BeautifulSoup(response.text, "html.parser")
    results = soup.find_all(id="listingsWrapperMainListing")
    list_head = soup.find_all("div", class_="listing-details")
    rcs = rcs+100
       
    for sp in list_head:
        tag = sp.find("span")
        linkTag = sp.find_all('a', href=True)
        carName = tag.text
        #carName = carName.replace(" ","")
        carName = carName.replace("\n","")
        carName = carName.replace("\r","")
        carTitle.append(carName[1:len(carName)-1].strip())
        for link in linkTag:
            linkUrl.append("autotrader.ca"+link['href'])
print("No. of resuls for CarTitle:",len(carTitle)," \n No. of Results for URL: ",len(linkUrl)) 


In [ ]:
d = {'Url':linkUrl}
df = pd.DataFrame(d)
#df.to_csv(path_or_buf="URL.csv", index = False)

Scraping Actual from the extracted URLs

In [ ]:
urlist = df['Url'].to_list()

In [ ]:


def getCarData(url_data):
    # location= ontario dealer = private
    base_url =   url_data

    ser = Service(r"c:\path\to\your\chromedriver.exe")
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('headless')

    # adding user agent to get headless mode running which blocked if user agent not mentioned
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    chrome_options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(service=ser, options=chrome_options)
    driver.get(base_url)

    #response = requests.get(base_url, headers = headers)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # getting and splitting car title element into list
    car_info = soup.find_all("h1", class_="hero-title")
    car_title = ""
    car_loc_list=""
 

    if(len(car_info)>0):
        car_title = car_info[0].text

    # getting and splitting car info which location,kms, car title 
    car_location = soup.find_all("p", class_="hero-location")
    
    if(len(car_location)>0):
        car_loc_list = car_location[0].text

    # getting all specs data from li
    Ul_list_head1 = soup.find("ul", id="sl-card-body")
    carSpecs = {}
    if Ul_list_head1 is not None:
        li_list_content = Ul_list_head1.find_all("li")
        
        carSpecs["Car Location"] = car_loc_list
        carSpecs["car title"] = car_title
        for sp in li_list_content:
            span_list = sp.find_all("span");
            carSpecsKey = span_list[0].text
            carSpecsValue = span_list[1].text
            carSpecs[carSpecsKey] = carSpecsValue
    
    return carSpecs

Calling the Fetch Function and adding to the database

In [ ]:
# importing module
from pymongo import MongoClient
  
# creation of MongoClient
client=MongoClient()
  
# Connect with the portnumber and host
client = MongoClient("mongodb://localhost:27017/")
  
# Access database
mydatabase = client['Price_Predictor']
  
# Access collection of the database
col=mydatabase.car_details
  
# dictionary to be added in the database
all_cars_data1 = []
for index,row in df.iterrows():
    carSpecs1 = getCarData('https://www.'+row['Url'].strip())
    # inserting the data in the database
    mydatabase.car_details.insert_one(carSpecs1)
    # print("Done",index)

In [ ]:
# cursor = col.find()
# for cars in cursor:
#     print(cars)